# Read some USGS 1m Terrain data in COG format

In [ ]:
import rioxarray as rxr
import hvplot.xarray
import fsspec

For public read buckets, we don't need credentials, but rasterio uses GDAL under the hood, and we need to tell GDAL not to use credentials by setting 
the `AWS_NO_SIGN_REQUEST` environment variable:

In [ ]:
import os
os.environ['GDAL_DISABLE_READDIR_ON_OPEN']='EMPTY_DIR' #This is KEY! otherwise we send a bunch of HTTP GET requests to test for common sidecar metadata
os.environ['AWS_NO_SIGN_REQUEST']='YES' #Since this is a public bucket, we don't need authentication
os.environ['GDAL_MAX_RAW_BLOCK_CACHE_SIZE']='200000000'  #200MB: Want this to be greater than size of uncompressed raster to overcome a 10 MB limit in the GeoTIFF driver for range request merging.
os.environ['GDAL_SWATH_SIZE']='200000000'  #also increase this if increasing MAX_RAW_BLOCK_CACHE_SIZE
os.environ['VSI_CURL_CACHE_SIZE']='200000000' #also increase this if increasing MAX_RAW_BLOCK_CACHE_SIZE

In [ ]:
fs = fsspec.filesystem('s3', anon=True)

In [ ]:
fs.glob('s3://prd-tnm/StagedProducts/Elevation/1m/Projects/ME*')

In [ ]:
fs.ls('prd-tnm/StagedProducts/Elevation/1m/Projects/ME_Eastern_B1_2017/TIFF/')

In [ ]:
s3uri='s3://prd-tnm/StagedProducts/Elevation/1m/Projects/ME_Eastern_B1_2017/TIFF/USGS_one_meter_x57y506_ME_Eastern_B1_2017.tif'

In [ ]:
da = rxr.open_rasterio(s3uri, masked=True).squeeze(dim='band', drop=True)

In [ ]:
da.spatial_ref.crs_wkt[-15:]

In [ ]:
da.hvplot.image(rasterize=True, cmap='turbo', crs='epsg:26919', tiles='OSM', 
                alpha=0.5)

In [ ]:
da.x[1].values - da.x[0].values